In [1]:
import pandas as pd #importing panadas

In [2]:
df = pd.read_csv('loan_data_set.csv') #using panadas to read the dataset

In [3]:
df.head() #printing the first 5 data entries of the dataset

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


Connection to azure SQL DB --> Note that this is NOT a secure way to connect to your Azure SQL DB - This is for personal use - see below how to use in production

In [4]:
from sqlalchemy import create_engine
import urllib #urllib is a built-in Python module used for working with URLs — it helps you fetch, parse, and manipulate URLs

#connection details
server = '' #enter your sql data server address ends with ".database.windows.net:
database = '' #enter your database name that you created in azure
username = '' #enter your admin user name you setup when configuring the sql server
password = '' #enter your pasword  you setup when configuring the sql server

In [5]:
#Build connection string with OBDC Driver 17
#download and install the ODBC driver 17 from the url below
# https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver17#download-for-windows

params = urllib.parse.quote_plus( #This encodes special characters in the string (e.g., spaces, =, ;) so the entire string can be passed safely as part of a URL.
    f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}' #using an f string to read our variables from above 
)
 
#crate SQLAlchemy engine
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}') #creates a SQLAlchemy engine object that lets your Python code connect to a Microsoft SQL Server database (like Azure SQL) using the pyodbc driver.

In [6]:
#name of the target table in your database
table_name = 'loan_dataset'

#upload data to Azure SqL - 'replace' will drop if it exists and create a new one
#as we are creating a new one we can leave it as replace

#A Pandas method that writes the DataFrame to a SQL database table.
#The name of the table in the database where the data will be stored.
#connection or engine object from SQLAlchemy - tells Pandas where to write the code
df.to_sql(table_name,con=engine,if_exists='replace',index=False) #Index = False - do not write dataframes index as a seperate column - if we do not set this we will need to remove this column after

print(f'Data successfully uploaded yo table {table_name}')

Data successfully uploaded yo table loan_dataset


**Secure Connection to Azure-SQL-DB using Azure Vault Keys**

In [8]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from sqlalchemy import create_engine
import urllib

In [ ]:
#Configure Azure Key vaults from the portal
#Create keys for the sql server, SQL DB, Username and Password
#On the overview page you will see Vault URI: you will add this here

key_vault_name = "Key vault name you created in the portal"
kv_uri = f"https://{key_vault_name}.vault.azure.net/" # this is the unique vault URI on the overview page

In [ ]:
#Create an instance of the class DefaultAzureCredential
credential = DefaultAzureCredential()
client = SecretClient(vault_url=kv_uri, credential=credential) #create the client object to use to get the stored passwords from Azure Key Vaults

In [ ]:
#using the get.secret method we can retrieve our passwords from Azure Key Vaults
sql_server = client.get_secret("Project1-SQL-Server").value
database = client.get_secret("Project1-SQL-DB").value
username = client.get_secret("Project1-SQL-Username").value
password = client.get_secret("Project1-SQL-Password").value

In [ ]:
#Build connection string with OBDC Driver 17
#download and install the ODBC driver 17 from the url below
# https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver17#download-for-windows

params = urllib.parse.quote_plus( #This encodes special characters in the string (e.g., spaces, =, ;) so the entire string can be passed safely as part of a URL.
    f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={sql_server};DATABASE={database};UID={username};PWD={password}' #using an f string to read our variables from above 
)
 
#crate SQLAlchemy engine
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}') #creates a SQLAlchemy engine object that lets your Python code connect to a Microsoft SQL Server database (like Azure SQL) using the pyodbc driver.

In [ ]:
#name of the target table in your database
table_name = 'loan_dataset'

#upload data to Azure SqL - 'replace' will drop if it exists and create a new one
#as we are creating a new one we can leave it as replace

#A Pandas method that writes the DataFrame to a SQL database table.
#The name of the table in the database where the data will be stored.
#connection or engine object from SQLAlchemy - tells Pandas where to write the code
df.to_sql(table_name,con=engine,if_exists='replace',index=False) #Index = False - do not write dataframes index as a seperate column - if we do not set this we will need to remove this column after

print(f'Data successfully uploaded yo table {table_name}')